In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2 as cv 
import glob
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler,Subset
from torchvision import transforms
from IPython.display import HTML
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torchvision  import datasets 
from sklearn.model_selection import KFold
import optuna
from optuna.trial import Trial
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Subset, DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
combined_data = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert to Tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])

In [5]:
combinedData = datasets.ImageFolder(combined_data, transform=transform)

In [6]:
combinedData

Dataset ImageFolder
    Number of datapoints: 10905
    Root location: /kaggle/input/deepfake-and-real-images/Dataset/Test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [7]:
kf = KFold(n_splits=3, shuffle=True, random_state=42) 

In [8]:
samples = len(combinedData)
samples

10905

In [9]:
folds = list(kf.split(np.arange(samples)))

In [10]:
folds

[(array([    1,     2,     4, ..., 10900, 10901, 10902]),
  array([    0,     3,     8, ..., 10899, 10903, 10904])),
 (array([    0,     1,     3, ..., 10902, 10903, 10904]),
  array([    2,    15,    18, ..., 10889, 10900, 10901])),
 (array([    0,     2,     3, ..., 10901, 10903, 10904]),
  array([    1,     4,     5, ..., 10896, 10897, 10902]))]

# Split our data into 3 Kfolds

In [11]:
for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
    print(f"Fold {fold_idx + 1}")
    
    train_size = int(0.75* len(train_val_idx)) 
    val_size = len(train_val_idx) - train_size #25
    
    train_idx = train_val_idx[:train_size]
    val_idx = train_val_idx[train_size:]
    
    # Create subsets
    train_set = Subset(combinedData, train_idx)
    val_set = Subset(combinedData, val_idx)
    test_set = Subset(combinedData, test_idx)
    
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
    
    print(f"Train size: {len(train_set)}")
    print(f"Validation size: {len(val_set)}")
    print(f"Test size: {len(test_set)}")

Fold 1
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 2
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 3
Train size: 5452
Validation size: 1818
Test size: 3635


# Define our model mobilenet

In [13]:
def get_model(name):
    if name == "mobilenet":
        model = models.mobilenet_v2(pretrained=True)
        # Freeze all parameters
        for param in model.parameters():
            param.requires_grad = False
        # Replace the classifier to output 2 classes
        model.classifier[1] = nn.Linear(model.last_channel, 2)
    else:
        raise ValueError("Model name must be 'mobilenet'")
    return model

# Training function

In [14]:
def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)  #loss function used to compute the error between predictions and true labels
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1) # max score from the scores of two classes
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc




# Eval function

In [15]:
def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Optimize the Learning rate using Optuna

In [16]:
def LR_optimization(trial: Trial):
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    
    # Iterate over folds
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"\nFold {fold_idx + 1}")
        
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        
        for model_name in ["mobilenet"]:
            print(f"\nTraining model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            # Initialize model
            model = get_model(model_name).to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                # Train and evaluate
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                val_loss, val_acc = evaluate(model, val_loader, criterion)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            _, val_acc = evaluate(model, val_loader, criterion)
            fold_results.append(val_acc) 

    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc



## Without Fine-tune 

In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(LR_optimization, n_trials=5)

best_trial = study.best_trial
print("\nBest Trial:")
print(f"  Value (Mean Validation Accuracy): {best_trial.value}")
print(f"  Params: {best_trial.params}")

[I 2024-12-24 19:00:36,737] A new study created in memory with name: no-name-a1fbde47-1a9d-4340-be93-dfbacba8b176
<ipython-input-16-fa3930ba039d>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-


Fold 1

Training model: MOBILENET on Fold 1


100%|██████████| 13.6M/13.6M [00:00<00:00, 160MB/s]



Epoch 1/5 | Learning Rate: 0.000484


Train Loss: 0.5996, Train Acc: 0.6820
Val Loss: 1.1021, Val Acc: 0.1700

Epoch 2/5 | Learning Rate: 0.000484


...

Train Loss: 0.5654, Train Acc: 0.7302
Val Loss: 0.6806, Val Acc: 0.6353

Epoch 4/5 | Learning Rate: 0.001875


Train Loss: 0.5206, Train Acc: 0.7436
Val Loss: 1.4591, Val Acc: 0.2305

Epoch 5/5 | Learning Rate: 0.001875


Train Loss: 0.5360, Train Acc: 0.7353
Val Loss: 0.5211, Val Acc: 0.7398


[I 2024-12-24 19:38:39,462] Trial 4 finished with value: 0.4961496149614962 and parameters: {'lr': 0.0018754078905789996}. Best is trial 1 with value: 0.7211221122112211.



Best Trial:
  Value (Mean Validation Accuracy): 0.7211221122112211
  Params: {'lr': 0.007232401612829719}


# After the Optuna optimization process, we will use the best trial to set the learning rate

In [18]:
# Best learning rate from Optuna
best_lr = 0.007232401612829719

In [19]:
final_results = {}

for model_name in ["mobilenet"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    # Initialize model
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    
    # Create DataLoaders (assuming combinedData is split into train and test datasets)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    # Train the model
    for epoch in range(10):  # Example: 10 epochs
        print(f"\nEpoch {epoch + 1}/10")
        
        # Train and evaluate
        train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    
    # Final evaluation on the test set
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    
    # Store results
    final_results[model_name] = {
        "Test Loss": test_loss,
        "Test Accuracy": test_acc
    }

# Print final results
print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"{model_name.upper()}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


Training model: MOBILENET with best learning rate 0.007232401612829719

Epoch 1/10


Train Loss: 0.7637, Train Acc: 0.6709

Epoch 2/10


Train Loss: 0.6161, Train Acc: 0.7117

Epoch 3/10


Train Loss: 0.6679, Train Acc: 0.7098

Epoch 4/10


Train Loss: 0.7015, Train Acc: 0.7062

Epoch 5/10


...

Train Loss: 0.7132, Train Acc: 0.7129

Epoch 8/10


Train Loss: 0.7121, Train Acc: 0.7129

Epoch 9/10


Train Loss: 0.8014, Train Acc: 0.7062

Epoch 10/10


Train Loss: 0.7198, Train Acc: 0.7115


Test Loss: 1.1034, Test Acc: 0.5827

Final Results:
MOBILENET:
  Test Loss: 1.1034
  Test Accuracy: 0.5827


In [20]:
# Function to evaluate the model and get performance metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Initialize cross-validation metrics storage
confusion_matrices = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

best_lr = 0.007232401612829719

for model_name in ["mobilenet"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"Fold {fold_idx + 1}")

        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
        
        
        model = get_model(model_name).to(device)
        optimizer = optim.Adam(model.parameters(), lr=best_lr)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        for epoch in range(10):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/10, Loss: {running_loss / len(train_loader):.4f}")

    
        preds, labels = evaluate_model(model, test_loader)

        
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average='weighted', zero_division=0)
        recall = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        cm = confusion_matrix(labels, preds)

        # Store metrics for the fold
        confusion_matrices.append(cm)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold_idx + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{cm}\n")

    # Average metrics across folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)

    # Print final average metrics for the model
    print(f"\nFinal Results for {model_name.upper()}:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1_score:.4f}")
    print(f"Average Confusion Matrix:\n{avg_confusion_matrix}")


Training model: MOBILENET with best learning rate 0.007232401612829719
Fold 1
Epoch 1/10, Loss: 0.7662
Epoch 2/10, Loss: 0.6707
Epoch 3/10, Loss: 0.7132
Epoch 4/10, Loss: 0.6329
Epoch 5/10, Loss: 0.6383
Epoch 6/10, Loss: 0.7477
Epoch 7/10, Loss: 0.7417
Epoch 8/10, Loss: 0.6694
Epoch 9/10, Loss: 0.8421
Epoch 10/10, Loss: 0.7209
Fold 1 - Accuracy: 0.7073, Precision: 0.7079, Recall: 0.7073, F1-Score: 0.7066
Confusion Matrix:
[[1399  465]
 [ 599 1172]]

Fold 2


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.7659
Epoch 2/10, Loss: 0.6756
Epoch 3/10, Loss: 0.7129
Epoch 4/10, Loss: 0.7410
Epoch 5/10, Loss: 0.6993
Epoch 6/10, Loss: 0.7726
Epoch 7/10, Loss: 0.7592
Epoch 8/10, Loss: 0.6633
Epoch 9/10, Loss: 0.6915
Epoch 10/10, Loss: 0.6897
Fold 2 - Accuracy: 0.7254, Precision: 0.7264, Recall: 0.7254, F1-Score: 0.7253
Confusion Matrix:
[[1281  553]
 [ 445 1356]]

Fold 3


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.7083
Epoch 2/10, Loss: 0.6950
Epoch 3/10, Loss: 0.6624
Epoch 4/10, Loss: 0.7241
Epoch 5/10, Loss: 0.7068
Epoch 6/10, Loss: 0.6557
Epoch 7/10, Loss: 0.6992
Epoch 8/10, Loss: 0.6905
Epoch 9/10, Loss: 0.7192
Epoch 10/10, Loss: 0.7413
Fold 3 - Accuracy: 0.6790, Precision: 0.7387, Recall: 0.6790, F1-Score: 0.6557
Confusion Matrix:
[[ 751 1043]
 [ 124 1717]]


Final Results for MOBILENET:
Average Accuracy: 0.7039
Average Precision: 0.7243
Average Recall: 0.7039
Average F1-Score: 0.6959
Average Confusion Matrix:
[[1143.66666667  687.        ]
 [ 389.33333333 1415.        ]]


## Fine-tuning 

## Applying Fine-tune

In [17]:
# Fine-tune
def get_model(name):
    if name == "mobilenet":
        # Load the pretrained MobileNet model
        model = models.mobilenet_v2(pretrained=True)
        
        # Freeze all layers initially
        for param in model.parameters():
            param.requires_grad = False
        
        # Fine-tune the last 5 layers
        # 1. Unfreeze the last 5 convolutional blocks in the feature extractor
        for param in model.features[-5:].parameters():
            param.requires_grad = True
        
        # 2. Modify the classifier for the desired number of output classes
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
        
        # 3. Ensure the classifier layer is trainable
        for param in model.classifier.parameters():
            param.requires_grad = True
        
        # Print the number of trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Total trainable parameters: {trainable_params:,}")
        
    else:
        raise ValueError("Model name must be 'mobilenet'")
    
    return model


## Training Function 

In [21]:
# Fine-tune

def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Add gradient clipping to prevent exploding gradients
    max_grad_norm = 1.0
    
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        
        # Clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [22]:
def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [23]:
# Fine-tune

def LR_optimization(trial: Trial):
    # Use a smaller learning rate range for fine-tuning
    lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
    
    # Rest of the function remains the same
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"\nFold {fold_idx + 1}")
        
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        for model_name in ["mobilenet"]:
            print(f"\nFine-tuning model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            model = get_model(model_name).to(device)
            criterion = nn.CrossEntropyLoss()
            
            # Use a smaller weight decay for fine-tuning
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            
            # Add learning rate scheduler for fine-tuning
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='max', factor=0.5, patience=2, verbose=True
            )

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                val_loss, val_acc = evaluate(model, val_loader, criterion)
                
                # Update learning rate based on validation accuracy
                scheduler.step(val_acc)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            _, val_acc = evaluate(model, val_loader, criterion)
            fold_results.append(val_acc)

    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc

In [24]:
study = optuna.create_study(direction="maximize")
study.optimize(LR_optimization, n_trials=5)

best_trial = study.best_trial
print("\nBest Trial:")
print(f"  Value (Mean Validation Accuracy): {best_trial.value}")
print(f"  Params: {best_trial.params}")

[I 2024-12-24 21:50:49,214] A new study created in memory with name: no-name-67461f74-e141-4a5b-aa41-ec2a28351a19
<ipython-input-23-400e3bc04bdc>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)



Fold 1

Fine-tuning model: MOBILENET on Fold 1
Total trainable parameters: 1,683,906

Epoch 1/5 | Learning Rate: 0.000085


Train Loss: 0.4322, Train Acc: 0.7801
Val Loss: 0.5501, Val Acc: 0.7118

Epoch 2/5 | Learning Rate: 0.000085


Train Loss: 0.2560, Train Acc: 0.8914
Val Loss: 0.6317, Val Acc: 0.7063

Epoch 3/5 | Learning Rate: 0.000085


Train Loss: 0.1644, Train Acc: 0.9321
Val Loss: 0.6713, Val Acc: 0.7387

Epoch 4/5 | Learning Rate: 0.000085


...

Train Loss: 0.6079, Train Acc: 0.6722
Val Loss: 0.8356, Val Acc: 0.2816

Epoch 4/5 | Learning Rate: 0.000001


Train Loss: 0.5802, Train Acc: 0.6911
Val Loss: 0.8471, Val Acc: 0.2921

Epoch 5/5 | Learning Rate: 0.000001


Train Loss: 0.5649, Train Acc: 0.7030
Val Loss: 0.8640, Val Acc: 0.2794


[I 2024-12-24 22:34:40,661] Trial 4 finished with value: 0.44994499449944997 and parameters: {'lr': 1.2424743744259802e-06}. Best is trial 1 with value: 0.7389072240557389.



Best Trial:
  Value (Mean Validation Accuracy): 0.7389072240557389
  Params: {'lr': 0.00018721286755151963}


# After the Optuna optimization process, we will use the best trial to set the learning rate

In [25]:
# Best learning rate from Optuna
best_lr = 0.0009289966381913214

In [26]:
final_results = {}

for model_name in ["mobilenet"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    # Initialize model
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    
    # Create DataLoaders (assuming combinedData is split into train and test datasets)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    # Train the model
    for epoch in range(10):  # Example: 10 epochs
        print(f"\nEpoch {epoch + 1}/10")
        
        # Train and evaluate
        train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    
    # Final evaluation on the test set
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    
    # Store results
    final_results[model_name] = {
        "Test Loss": test_loss,
        "Test Accuracy": test_acc
    }

# Print final results
print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"{model_name.upper()}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


Training model: MOBILENET with best learning rate 0.0009289966381913214
Total trainable parameters: 1,683,906

Epoch 1/10


Train Loss: 0.3825, Train Acc: 0.8309

Epoch 2/10


Train Loss: 0.2185, Train Acc: 0.9136

Epoch 3/10


Train Loss: 0.1644, Train Acc: 0.9384

Epoch 4/10


Train Loss: 0.1157, Train Acc: 0.9578

Epoch 5/10


...

Train Loss: 0.0514, Train Acc: 0.9842

Epoch 8/10


Train Loss: 0.0335, Train Acc: 0.9861

Epoch 9/10


Train Loss: 0.0424, Train Acc: 0.9873

Epoch 10/10


Train Loss: 0.0268, Train Acc: 0.9910


Test Loss: 0.5699, Test Acc: 0.8946

Final Results:
MOBILENET:
  Test Loss: 0.5699
  Test Accuracy: 0.8946


In [27]:
# Function to evaluate the model and get performance metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Initialize cross-validation metrics storage
confusion_matrices = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

best_lr = 0.0009289966381913214

for model_name in ["mobilenet"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"Fold {fold_idx + 1}")

        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
        
        
        model = get_model(model_name).to(device)
        optimizer = optim.Adam(model.parameters(), lr=best_lr)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        for epoch in range(10):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/10, Loss: {running_loss / len(train_loader):.4f}")

    
        preds, labels = evaluate_model(model, test_loader)

        
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average='weighted', zero_division=0)
        recall = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        cm = confusion_matrix(labels, preds)

        # Store metrics for the fold
        confusion_matrices.append(cm)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold_idx + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{cm}\n")

    # Average metrics across folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)

    # Print final average metrics for the model
    print(f"\nFinal Results for {model_name.upper()}:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1_score:.4f}")
    print(f"Average Confusion Matrix:\n{avg_confusion_matrix}")


Training model: MOBILENET with best learning rate 0.0009289966381913214
Fold 1
Total trainable parameters: 1,683,906
Epoch 1/10, Loss: 0.3863
Epoch 2/10, Loss: 0.2188
Epoch 3/10, Loss: 0.1494
Epoch 4/10, Loss: 0.1101
Epoch 5/10, Loss: 0.0855
Epoch 6/10, Loss: 0.0826
Epoch 7/10, Loss: 0.0491
Epoch 8/10, Loss: 0.0340
Epoch 9/10, Loss: 0.0328
Epoch 10/10, Loss: 0.0334
Fold 1 - Accuracy: 0.8825, Precision: 0.8863, Recall: 0.8825, F1-Score: 0.8821
Confusion Matrix:
[[1744  120]
 [ 307 1464]]

Fold 2
Total trainable parameters: 1,683,906


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.3924
Epoch 2/10, Loss: 0.2251
Epoch 3/10, Loss: 0.1665
Epoch 4/10, Loss: 0.1297
Epoch 5/10, Loss: 0.0815
Epoch 6/10, Loss: 0.0771
Epoch 7/10, Loss: 0.0539
Epoch 8/10, Loss: 0.0746
Epoch 9/10, Loss: 0.0529
Epoch 10/10, Loss: 0.0325
Fold 2 - Accuracy: 0.8751, Precision: 0.8751, Recall: 0.8751, F1-Score: 0.8751
Confusion Matrix:
[[1618  216]
 [ 238 1563]]

Fold 3
Total trainable parameters: 1,683,906


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.3868
Epoch 2/10, Loss: 0.2251
Epoch 3/10, Loss: 0.1677
Epoch 4/10, Loss: 0.1083
Epoch 5/10, Loss: 0.0781
Epoch 6/10, Loss: 0.0614
Epoch 7/10, Loss: 0.0600
Epoch 8/10, Loss: 0.0399
Epoch 9/10, Loss: 0.0457
Epoch 10/10, Loss: 0.0338
Fold 3 - Accuracy: 0.8957, Precision: 0.9006, Recall: 0.8957, F1-Score: 0.8955
Confusion Matrix:
[[1703   91]
 [ 288 1553]]


Final Results for MOBILENET:
Average Accuracy: 0.8845
Average Precision: 0.8874
Average Recall: 0.8845
Average F1-Score: 0.8842
Average Confusion Matrix:
[[1688.33333333  142.33333333]
 [ 277.66666667 1526.66666667]]
